In [1]:
import os
import librosa
import random
import json
import numpy as np

In [2]:
def crearDiccionarioVacio():
    return {"ID": [],"MFCCDD": [], "ZCR": []}

In [3]:
def crearJson(nombreArchivo, datos):
    with open(nombreArchivo, "w") as fp:
        json.dump(datos, fp, indent=4)
    print("Archivo json creado")

In [4]:
def partirAudio(signal, longitudPartes, tam_ventana):
    partes = []
    # Inicio de ms en el segmento de audio
    inicio = 0
    # Fin de ms en el segmento de audio
    fin = longitudPartes
    # Si fin < longitu del audio, se puede seguir partiendo en trozos
    while(fin<len(signal)):
        partes.append(signal[inicio:fin])
        
        inicio = fin-tam_ventana # Se reestablecen los valores de ms de inicio y fin del segmento
        fin = fin+longitudPartes-tam_ventana
    
    if len(signal)-inicio > longitudPartes/2: # Si al terminar queda algo de audio aprovechable, se crea un ultimo segmento
        partes.append(signal[len(signal)-longitudPartes-tam_ventana:len(signal)-tam_ventana])

    return partes

In [5]:
def extraerFeatures(clientes, ruta, inicioSignal, cantidadSamples, hop_length, n_mfcc, n_fft):

    diccionario = crearDiccionarioVacio()
    longitudTrozos = int(inicioSignal/2) # le doy 0.25 seg en este caso

    contador = 1

    for cliente in clientes:

        for audio in os.listdir(os.path.join(ruta, cliente)):

            signal, sr = librosa.load(os.path.join(ruta, cliente, audio))

            #Se elimina el 'silencio'
            signal, _ = librosa.effects.trim(signal)

            trozos = partirAudio(signal, longitudTrozos, int(cantidadSamples/10)) # se parte en trozos de 0.25seg cada uno con una ventana de 10ms   (antes era /5 para 20 ms)

            for trozo in trozos: # se recorre cada uno de los segementos del archivo de audio obtenidos antes 
                # extraccion zero crossing rate
                zcr = librosa.feature.zero_crossing_rate(trozo)
                
                # extraccion de MFCCs de cada segmento del archivo de audio
                mfccs = librosa.feature.mfcc(y=trozo, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
                
                delta_mfccs = librosa.feature.delta(mfccs) # obtencion primer delta de los mfcc
                delta2_mfccs = librosa.feature.delta(mfccs, order=2) # obtencion segundo delta
                mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))

                diccionario['ID'].append(cliente)#.split("_")[0])
                diccionario["MFCCDD"].append(mfccs_features.T.tolist())
                diccionario["ZCR"].append(zcr.tolist())

                        
        print(f'{contador}: {cliente}', end="\r")
        contador += 1

    print("Diccionario creado")
    return diccionario

In [6]:
def cantidadAudiosFinalesSexo(clientes, ruta, categoriaAudios):
    cant = []
    cantidadMujeres = 0
    cantidadHombres = 0

    for c in clientes:
        cant.append((len(os.listdir(os.path.join(ruta, c))), c.split("_")[1]))


    for c in cant:
        if c[1] == 'H':
            cantidadHombres+=c[0]
        else:
            cantidadMujeres+=c[0]
    
    print(f'Una vez contados los audios de {categoriaAudios}, hay {cantidadHombres} audios de hombres y {cantidadMujeres} de mujeres')

In [8]:
rutaEntrenamiento = '../audios/seleccionEntrenamiento'
rutaRefuerzo = '../audios/seleccionRefuerzo'

UN_SEGUNDO_SAMPLES = 22050
inicioSignal = int(UN_SEGUNDO_SAMPLES/2) # medio segundo
jsonTrainFile1 = "..\\Json\\espTrainData1.json"
jsonTrainFile2 = "..\\Json\\espTrainData2.json"
jsonTrainFile3 = "..\\Json\\espTrainData3.json"

jsonRefFile1 = "..\\Json\\espRefData1.json"
jsonRefFile2 = "..\\Json\\espRefData2.json"
jsonRefFile3 = "..\\Json\\espRefData3.json"

n_mfcc = 13
hop_length = 512
n_fft= 2048

clientesEntrenamiento = os.listdir(rutaEntrenamiento)
clientesRefuerzo = os.listdir(rutaRefuerzo)

In [9]:
cantidadAudiosFinalesSexo(clientesEntrenamiento, rutaEntrenamiento, "entrenamiento")
cantidadAudiosFinalesSexo(clientesRefuerzo, rutaRefuerzo, "refuerzo")

Una vez contados los audios de entrenamiento, hay 4500 audios de hombres y 3470 de mujeres
Una vez contados los audios de refuerzo, hay 1110 audios de hombres y 1045 de mujeres


In [10]:
# Primera parte de entrenamiento ( lo parto porque sino el json pesa demasiado para cargarlo en google colab)
trainDict1 = extraerFeatures(clientesEntrenamiento[:int(len(clientesEntrenamiento)/3)], rutaEntrenamiento, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonTrainFile1, trainDict1)
del trainDict1

Diccionario creado
Archivo json creado


In [11]:
# Segunda parte entrenamiento
trainDict2 = extraerFeatures(clientesEntrenamiento[int(len(clientesEntrenamiento)/3):(int(len(clientesEntrenamiento)/3)*2)], rutaEntrenamiento, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonTrainFile2, trainDict2)
del trainDict2

Diccionario creado
Archivo json creado


In [12]:
# Tercera parte entrenamiento
trainDict3 = extraerFeatures(clientesEntrenamiento[(int(len(clientesEntrenamiento)/3)*2):], rutaEntrenamiento, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonTrainFile3, trainDict3)
del trainDict3

Diccionario creado
Archivo json creado


In [13]:
# Primera parte refuerzo
refDict1 = extraerFeatures(clientesRefuerzo[:int(len(clientesRefuerzo)/3)], rutaRefuerzo, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonRefFile1, refDict1)
del refDict1

Diccionario creado
Archivo json creado


In [14]:
# Segunda parte refuerzo
refDict2 = extraerFeatures(clientesRefuerzo[int(len(clientesRefuerzo)/3):(int(len(clientesRefuerzo)/3)*2)], rutaRefuerzo, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonRefFile2, refDict2)
del refDict2

Diccionario creado
Archivo json creado


In [15]:
# Tercera parte refuerzo
refDict3 = extraerFeatures(clientesRefuerzo[(int(len(clientesRefuerzo)/3)*2):], rutaRefuerzo, inicioSignal, UN_SEGUNDO_SAMPLES, hop_length, n_mfcc, n_fft)
crearJson(jsonRefFile3, refDict3)
del refDict3

Diccionario creado
Archivo json creado
